In [1]:
# -------- CẤU HÌNH CHẠY LẠI CHO CÁC PART BỊ THIẾU --------

# Số part ban đầu khi chia full dataset
ORIGINAL_NUM_PARTS = 53

# Những part đã bị ngắt ở lần chạy đầu
MISSING_PARTS = [8, 15, 17, 39, 45, 46]

# Chia lại toàn bộ tập "thiếu" thành bao nhiêu part mới
NUM_PARTS = 12              # ở đây là 12

# Mỗi lần submit notebook trên Kaggle, đổi NUM_ID = 1..12
NUM_ID = 8                  # <-- sửa số này khi chạy từng part

BATCH_SIZE = 32             # số ảnh xử lý đồng thời trên mỗi GPU

# Đường dẫn đến file index (giữ nguyên)
INDEX_FILE = "/kaggle/input/irkeyframeindex/keyframe_index_kaggle.json"

In [2]:
import os
import json
import gc
import torch
from PIL import Image
from tqdm import tqdm
from transformers import AutoModelForCausalLM
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

OUTPUT_DIR = "/kaggle/working/captions_per_part"
os.makedirs(OUTPUT_DIR, exist_ok=True)

MODEL_NAME = "vikhyatk/moondream2"
REVISION = "2025-06-21"

# Lock để đồng bộ khi ghi kết quả
caption_lock = Lock()


def load_model(device):
    print(f"📥 Loading Moondream2 model on {device}...")
    device_id = device.index if hasattr(device, 'index') else int(str(device).split(':')[-1])
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        revision=REVISION,
        trust_remote_code=True,
        device_map={"": device_id}
    )
    print(f"✅ Model loaded on {device}")
    return model


def caption_image(model, image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        result = model.caption(image, length="normal")
        return result["caption"]
    except Exception as e:
        print(f"⚠️ Error captioning {image_path}: {e}")
        return ""


def load_index_file(index_path):
    """Load all images from index file as a flat list"""
    with open(index_path, 'r') as f:
        index_data = json.load(f)
    
    # Convert to list of (idx, img_path) tuples
    all_images = [(int(idx), img_path) for idx, img_path in index_data.items()]
    all_images.sort(key=lambda x: x[0])  # Sort by index
    
    return all_images


def split_dataset(all_images, num_parts, part_id):
    """Split dataset into parts and return the selected part"""
    total_images = len(all_images)
    
    # Calculate partition boundaries
    images_per_part = total_images // num_parts
    remainder = total_images % num_parts
    
    # Calculate start and end indices for this part
    if part_id <= remainder:
        start_idx = (part_id - 1) * (images_per_part + 1)
        end_idx = start_idx + images_per_part + 1
    else:
        start_idx = remainder * (images_per_part + 1) + (part_id - remainder - 1) * images_per_part
        end_idx = start_idx + images_per_part
    
    selected_images = all_images[start_idx:end_idx]
    
    print(f"📊 Total images: {total_images}")
    print(f"📦 Part {part_id}/{num_parts}: Processing images {start_idx+1} to {end_idx}")
    print(f"🖼️  Number of images in this part: {len(selected_images)}")
    
    return selected_images


def process_batch_on_gpu(model, image_batch, gpu_id, all_captions):
    """Process a batch of images on a specific GPU"""
    batch_captions = {}
    
    for idx, img_path in image_batch:
        caption = caption_image(model, img_path)
        if caption:
            batch_captions[str(idx)] = caption
    
    # Thread-safe update of shared results
    with caption_lock:
        all_captions.update(batch_captions)
    
    return len(batch_captions), gpu_id


def main():
    num_gpus = torch.cuda.device_count()
    print(f"🚀 Starting captioning on {num_gpus} GPU(s)")
    print(f"📋 Original dataset was split into {ORIGINAL_NUM_PARTS} parts")
    print(f"📋 Missing original parts: {MISSING_PARTS}")
    print(f"📋 New subset partition: Part {NUM_ID}/{NUM_PARTS}")
    print(f"📦 Batch size: {BATCH_SIZE} images per GPU")
    
    # 1. Load full index
    print(f"\n📂 Loading index file: {INDEX_FILE}")
    all_images = load_index_file(INDEX_FILE)
    
    # 2. Gom ảnh từ 6 part cũ bị thiếu thành một tập mới
    print(f"\n🧩 Collecting images from missing parts...")
    missing_images = []
    for missing_id in MISSING_PARTS:
        print(f"\n🔍 Collecting images from original part {missing_id}/{ORIGINAL_NUM_PARTS}...")
        part_images = split_dataset(all_images, ORIGINAL_NUM_PARTS, missing_id)
        print(f"   ➕ Added {len(part_images)} images")
        missing_images.extend(part_images)
    
    print(f"\n🧩 Total images in missing subset: {len(missing_images)}")
    print(f"📋 Re-splitting missing subset into {NUM_PARTS} new parts...")
    
    # 3. Chia lại tập "thiếu" thành 12 phần mới và lấy part đang cần chạy
    selected_images = split_dataset(missing_images, NUM_PARTS, NUM_ID)
    
    if not selected_images:
        print("❌ No images found in this partition!")
        return
    
    print(f"\n🖼️  Processing {len(selected_images)} images in this partition")
    
    # 4. Load model(s)
    print(f"\n{'='*60}")
    print("📥 Loading model(s)...")
    print(f"{'='*60}")
    
    models = []
    for gpu_id in range(num_gpus):
        device = torch.device(f"cuda:{gpu_id}")
        model = load_model(device)
        models.append(model)
    
    all_captions = {}
    
    print(f"\n{'='*60}")
    print(f"🔄 Processing images across {num_gpus} GPUs in parallel...")
    print(f"{'='*60}")
    
    # 5. Process images in batches across GPUs in parallel
    batch_idx = 0
    batch_num = 0
    
    with ThreadPoolExecutor(max_workers=num_gpus) as executor:
        while batch_idx < len(selected_images):
            batch_num += 1
            futures = []
            
            print(f"\n🔄 Batch {batch_num}")
            
            # Submit tasks for each GPU
            for gpu_id in range(num_gpus):
                start_idx = batch_idx + gpu_id * BATCH_SIZE
                end_idx = min(start_idx + BATCH_SIZE, len(selected_images))
                
                if start_idx >= len(selected_images):
                    break
                
                image_batch = selected_images[start_idx:end_idx]
                
                print(
                    f"   GPU {gpu_id}: Submitting {len(image_batch)} images "
                    f"(indices {start_idx+1}-{end_idx})"
                )
                
                future = executor.submit(
                    process_batch_on_gpu,
                    models[gpu_id],
                    image_batch,
                    gpu_id,
                    all_captions
                )
                futures.append(future)
            
            # Wait for all GPUs to complete this batch
            for future in as_completed(futures):
                num_captions, gpu_id = future.result()
                print(f"   ✅ GPU {gpu_id}: Generated {num_captions} captions")
            
            # Move to next batch (all GPUs combined)
            batch_idx += num_gpus * BATCH_SIZE
    
    # 6. Save results
    output_path = f"{OUTPUT_DIR}/captions_missing_part_{NUM_ID}_of_{NUM_PARTS}.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(all_captions, f, indent=2, ensure_ascii=False)
    
    print(f"\n{'='*60}")
    print("📊 FINAL SUMMARY (MISSING SUBSET)")
    print(f"New part {NUM_ID}/{NUM_PARTS} (from original parts {MISSING_PARTS})")
    print("="*60)
    print(f"✅ Saved captions: {output_path}")
    print(f"📊 Total images processed: {len(selected_images)}")
    print(f"📊 Total captions generated: {len(all_captions)}")
    print(f"📊 Success rate: {len(all_captions)/len(selected_images)*100:.1f}%")
    print("✅ All done!")
    
    # 7. Cleanup
    print(f"\n🧹 Cleaning up...")
    for gpu_id, model in enumerate(models):
        del model
        torch.cuda.empty_cache()
        if torch.cuda.is_available():
            torch.cuda.synchronize(gpu_id)
    gc.collect()
    print("✅ Cleanup complete")